In [2]:
# Seasonal ARIMA model of review counts
import sys
import pandas as pd
import numpy as np
np.warnings.filterwarnings("ignore")
from pandas.core import datetools
import itertools
import math
import datetime
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt

import statsmodels.api as sm  
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

import sklearn.linear_model
from statsmodels.tsa.stattools import adfuller



In [3]:
# load in the review data table
PGH_review = pd.read_csv('Yelp_ReviewCount.csv', sep = '\t')

mask = (PGH_review['Month'] >= '2010') & (PGH_review['Month'] <= '2017-11-01')
PGH_2010_2017 = PGH_review[mask]
PGH_2010_2017.set_index('Month',inplace=True)
PGH_2010_2017 = PGH_2010_2017.drop(['Uncategorized'], axis=1)

In [4]:
# define a function for grid_search arima parameters by minizing the error in test dataset
def grid_search_pdq(X):
    # Use data before 2016-12-01 as train data and evaluate error on 2017 data to find model parameters
    X_train = X.Value_log[:'2016-12-01']
    X_test = X.Value['2017-01-01':]

    # Use gridsearch to search for (p,d,q) and (P,D,Q) for Seasonal-ARIMA model 
    d = [0, 1]
    p = range(0, 3)
    q = range(0, 3)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

    # Grid_search
    best_pdq = None
    best_seasonal_pdq = None
    tmp_model = None
    best_mdl = None
    
    predictions = list()
    min_error = np.inf
    
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            tmp_mdl = sm.tsa.statespace.SARIMAX(X_train,
                                               order = param,
                                               seasonal_order = param_seasonal,
                                               enforce_stationarity = True,
                                               enforce_invertibility = True)
            try:
                #res = tmp_mdl.fit(maxiter=100, method='powell')
                res = tmp_mdl.fit()
            except:
                #print("Unexpected error occurred", sys.exc_info()[0])
                continue
                
            pred_uc = res.get_forecast(steps=11) 
            y_hat = np.exp(pred_uc.predicted_mean) - 1
            #error = mean_squared_error(X_test, y_hat)
            error = ((y_hat - X_test) ** 2).mean()

            if error < min_error:
                min_error = error
                best_pdq = param
                best_seasonal_pdq = param_seasonal
                best_mdl = tmp_mdl
       
    return best_pdq, best_seasonal_pdq

In [5]:
# loop through each column and perform time series analysis to make predictions for 2018.
import time
from tqdm import tqdm

predict_mse = []
forecast_list = []
for col in tqdm(list(PGH_2010_2017.columns)):
#for col in tqdm(list(PGH_2010_2017.columns)[:1]):
    print(col)
    review = PGH_2010_2017[[col]].copy()
    review.columns = ['Value']
    review.reset_index(inplace = True)
    review['Month'] = pd.to_datetime(review['Month'])
    review = review.set_index('Month')
    review.index = pd.DatetimeIndex(review.index.values,
                               freq=review.index.inferred_freq)
    
    
    review['Value_log']= review.Value.apply(lambda x: np.log(x+1))  
    review['log_first_difference'] = review.Value_log - review.Value_log.shift(1)  
#    result = adfuller(review.log_first_difference.dropna(inplace=False))
#     if result[1] >= 0.05:
#         print(col, result[1]) 

    best_pdq, best_seasonal_pdq = grid_search_pdq(review)
    
    print('best_pdq:', 'best_seasonal_pdq:')
    print(best_pdq, best_seasonal_pdq)
    
    # Use the grid_searched best_pdq, and best_seasonal_pdq to train the model
    #X_train = review.Value_log[:'2016-12-01']
    
    #try:
    #    mod = sm.tsa.statespace.SARIMAX(X_train, order = best_pdq, seasonal_order = best_seasonal_pdq)   
    #except TypeError:
    #    continue     
    #results = mod.fit()
    
    # get forecast 11 steps ahead in future, to compare with the test data set.
    #pred_uc = results.get_forecast(steps=11) 
    # get confidence intervals of forecasts
    #pred_ci = pred_uc.conf_int(alpha = 0.5)
    #y_hat = np.exp(pred_uc.predicted_mean) - 1
    #y_true = CBD.Value_log['2017-01-01':]
    #y_true = review.Value['2017-01-01':]

    # compute the mean square error
    #mse = ((y_hat - y_true) ** 2).mean()
    #predict_mse.append(mse)

    # Now model using all the data and predict 2017-12 to 2018-12
    try:
        sarima_model = sm.tsa.statespace.SARIMAX(review.Value_log, order = best_pdq, seasonal_order = best_seasonal_pdq)
        sarima_results = sarima_model.fit()
    except: 
        continue
    
    pred_uc = sarima_results.get_forecast(steps=13) 

    # get confidence intervals of forecasts
    pred_ci = pred_uc.conf_int(alpha = 0.5)
    rev_predict = round(np.exp(pred_uc.predicted_mean) - 1)
    lower_limit = round(np.exp(pred_ci.iloc[:, 0]) - 1)
    upper_limit = round(np.exp(pred_ci.iloc[:, 1]) - 1)

    # append predictions and confidence intervals to a list. 
    frames = [rev_predict, lower_limit, upper_limit]
    forecast = pd.concat(frames, axis = 1)
    forecast.columns = [col, col+' lower', col+' upper']
    forecast_list.append(forecast)

  0%|          | 0/68 [00:00<?, ?it/s]

Allegheny Center
best_pdq:

  1%|▏         | 1/68 [01:12<1:20:48, 72.37s/it]

 best_seasonal_pdq:
(1, 1, 0) (2, 0, 0, 12)
Allegheny West
best_pdq:

  3%|▎         | 2/68 [02:01<1:06:59, 60.90s/it]

...

 best_seasonal_pdq:
(1, 1, 2) (1, 0, 1, 12)
West End
best_pdq:

 99%|█████████▊| 67/68 [58:58<00:52, 52.82s/it]

 best_seasonal_pdq:
(0, 0, 2) (1, 0, 0, 12)
Westwood
best_pdq:

100%|██████████| 68/68 [59:35<00:00, 52.59s/it]

 best_seasonal_pdq:
(0, 1, 2) (1, 0, 0, 12)


In [6]:
# Save the data 
forecast_df = pd.concat(forecast_list, axis = 1)
forecast_df.to_csv("forecast_dataframe_50conf.csv", sep = '\t', index_label='Date')

In [15]:
# combine the historical and predicted data
PGH_review.set_index('Month', inplace=True)
frames = [PGH_review[:-1], forecast_df]
result = pd.concat(frames, join = 'inner')
result.index = pd.to_datetime(result.index)
result.to_csv('Yelp_ReviewCount_withPredict.csv',sep ='\t',index_label='Month')